In [2]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

ModuleNotFoundError: No module named 'cv2'

In [3]:
def unet_model(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same')(conv9)

    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [4]:
import glob

def load_data(image_folder, mask_folder):
    image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))
    mask_files = sorted(glob.glob(os.path.join(mask_folder, '*.png')))
    images = []
    masks = []

    for img_file, mask_file in zip(image_files, mask_files):
        image = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(mask_file, cv2.IMREAD_UNCHANGED)

        if image is None or mask is None:
            print(f"Failed to load image or mask: {img_file}, {mask_file}")
            continue

        if image.shape[:2] != mask.shape[:2]:
            print(f"Image and mask sizes do not match: {img_file}, {mask_file}")
            continue

        images.append(image)
        masks.append(mask)

    masks = np.array(masks) / 255.0
    images = np.array(images)

    return images, masks


# def load_data(image_folder, mask_folder, crop_size):
#     image_files = sorted(os.listdir(image_folder))
#     mask_files = sorted(os.listdir(mask_folder))
#     images = []
#     masks = []

#     for img_file, mask_file in zip(image_files, mask_files):
#         img_path = os.path.join(image_folder, img_file)
#         mask_path = os.path.join(mask_folder, mask_file)

#         # Load images and masks
#         image = cv2.imread(img_path)
#         mask = cv2.imread(mask_path, 0)

#         if image is None or mask is None:
#             print(f"Failed to load image or mask: {img_file}, {mask_file}")
#             continue

#         # Check if image and mask sizes match
#         if image.shape[:2] != mask.shape[:2]:
#             print(f"Image and mask sizes do not match: {img_file}, {mask_file}")
#             continue

#         # Crop image and mask
#         cropped_images = []
#         cropped_masks = []
#         h, w = image.shape[:2]
#         for i in range(0, h - crop_size + 1, crop_size):
#             for j in range(0, w - crop_size + 1, crop_size):
#                 cropped_image = image[i:i+crop_size, j:j+crop_size]
#                 cropped_mask = mask[i:i+crop_size, j:j+crop_size]
#                 cropped_images.append(cropped_image)
#                 cropped_masks.append(cropped_mask)

#         images.extend(cropped_images)
#         masks.extend(cropped_masks)

#     images = np.array(images)
#     masks = np.array(masks) / 255.0  # Normalize masks to values between 0 and 1

#     return images, masks

# 병렬 처리를 통해 이미지를 더 빨리 부르기 해

# import multiprocessing
# from functools import partial

# def load_image(image_folder, img_file):
#     img_path = os.path.join(image_folder, img_file)
#     image = cv2.imread(img_path)
#     return image

# def load_mask(mask_folder, mask_file):
#     mask_path = os.path.join(mask_folder, mask_file)
#     mask = cv2.imread(mask_path, 0)
#     return mask

# def load_data(image_folder, mask_folder, crop_size):
#     image_files = sorted(os.listdir(image_folder))
#     mask_files = sorted(os.listdir(mask_folder))

#     with multiprocessing.Pool() as pool:
#         images = pool.starmap(partial(load_image, image_folder), [(image_folder, img_file) for img_file in image_files])
#         masks = pool.starmap(partial(load_mask, mask_folder), [(mask_folder, mask_file) for mask_file in mask_files])


#     images = [image for image in images if image is not None]
#     masks = [mask for mask in masks if mask is not None]

#     # Crop image and mask
#     cropped_images = []
#     cropped_masks = []
#     for image, mask in zip(images, masks):
#         h, w = image.shape[:2]
#         for i in range(0, h - crop_size + 1, crop_size):
#             for j in range(0, w - crop_size + 1, crop_size):
#                 cropped_image = image[i:i+crop_size, j:j+crop_size]
#                 cropped_mask = mask[i:i+crop_size, j:j+crop_size]
#                 cropped_images.append(cropped_image)
#                 cropped_masks.append(cropped_mask)

#     cropped_images = np.array(cropped_images)
#     cropped_masks = np.array(cropped_masks) / 255.0  # Normalize masks to values between 0 and 1

#     return cropped_images, cropped_masks



In [5]:
from tensorflow.keras.utils import to_categorical

# 데이터 로드
train_image_folder = "C:\\Users\\IT\\Desktop\\ai\\train_img"
train_mask_folder = "C:\\Users\\IT\\Desktop\\ai\\train_mask"
test_image_folder = "C:\\Users\\IT\\Desktop\\ai\\test_img"

train_images, train_masks = load_data(train_image_folder, train_mask_folder, crop_size=224)
test_images, _ = load_data(test_image_folder, test_image_folder, crop_size=224)

# 타깃 데이터를 원-핫 인코딩
train_masks = to_categorical(train_masks, num_classes=2)

# U-Net 모델 생성
input_shape = (224, 224, 3)  # Updated input shape to match resized train images and masks
model = unet_model(input_shape)

# 모델 학습
model.fit(train_images, train_masks, batch_size=3, epochs=2)  # Removed reshape step

# 모델 저장
model.save("./train_model")

# # 테스트 데이터 예측
# resized_test_images = []
# for image in test_images:
#     resized_image = cv2.resize(image, (224, 224))
#     resized_test_images.append(resized_image)
# resized_test_images = np.array(resized_test_images)

# predictions = model.predict(resized_test_images)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/gdrive/My Drive/version/train_img'

In [ ]:
# import os
# import numpy as np
# import cv2
# from tensorflow.keras.models import load_model
# from google.colab.patches import cv2_imshow

# # 학습된 모델 불러오기
# model = load_model("./train_model")

# # 테스트 이미지 폴더 경로
# test_image_folder = '/content/gdrive/My Drive/version/test_img'

# # 결과 이미지 저장 폴더 경로
# output_folder = '/content/gdrive/My Drive/version/output'

# # 결과 이미지 저장 폴더가 없다면 생성
# os.makedirs(output_folder, exist_ok=True)

# # 테스트 이미지 파일 목록
# test_image_files = sorted(os.listdir(test_image_folder))

# # 테스트 이미지 예측 및 저장
# for img_file in test_image_files:
#     img_path = os.path.join(test_image_folder, img_file)

#     # 테스트 이미지 로드 및 크기 조정
#     image = cv2.imread(img_path)
#     resized_image = cv2.resize(image, (1024, 1024))

#     # 예측
#     prediction = model.predict(np.expand_dims(resized_image, axis=0))[0]
#     pred_mask = (prediction > 0.5).astype(np.uint8) * 255

#     # 예측 이미지 크기 조정
#     resized_pred_mask = cv2.resize(pred_mask, (224, 224))

#     # 결과 이미지 저장
#     output_path = os.path.join(output_folder, f"prediction_{img_file}")
#     cv2.imwrite(output_path, resized_pred_mask)

#     # 시각화 (일부 이미지만)
#     cv2_imshow(pred_mask)
#     cv2.waitKey(0)

# cv2.destroyAllWindows()

import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

# 학습된 모델 불러오기
model = load_model("./train_model")

# 테스트 이미지 폴더 경로
test_image_folder = '/content/gdrive/My Drive/version/test_img'

# 결과 이미지 저장 폴더 경로
output_folder = '/content/gdrive/My Drive/version/output5'

# 결과 이미지 저장 폴더가 없다면 생성
os.makedirs(output_folder, exist_ok=True)

# 테스트 이미지 파일 목록
test_image_files = sorted(os.listdir(test_image_folder))

# 테스트 이미지 예측 및 저장
for img_file in test_image_files:
    img_path = os.path.join(test_image_folder, img_file)

    # 테스트 이미지 로드 및 크기 조정
    image = cv2.imread(img_path)
    resized_image = cv2.resize(image, (224, 224))

    # 예측
    prediction = model.predict(np.expand_dims(resized_image, axis=0))[0]
    pred_mask = (prediction > 0.5).astype(np.uint8) * 255

    # 결과 이미지 저장
    output_path = os.path.join(output_folder, f"prediction_{img_file}")
    cv2.imwrite(output_path, pred_mask)

    # 시각화 (일부 이미지만)
    cv2_imshow(pred_mask)
    cv2.waitKey(0)

cv2.destroyAllWindows()